## Note

This notebook, **populate_database**, is designed to populate the **DuckDB database** and includes utility functions for performing sanity checks to ensure that the database is populated correctly.

### Important

The SQLite database referenced below is deprecated, and the new database is in DuckDB format. The link to the SQLite database is provided for reference only.

#### Usage

- Only execute the cells below if you need to:
  - Build the database from scratch
  - Download new annual data

- If you are downloading new data, please ensure that the database is placed in the following folder:  
  `../zip_data/duck_db_manager/database/us_naics_census_data_{year}.duckdb`

You can download the deprecated SQLite database from [this link](https://drive.google.com/file/d/1J1TLVkvnEQvX31MxdF2df04BNpiaExyf/view?usp=sharing).

### Reminder

**Caution**: Constructing the database from scratch may take several hours!

Here are the example running times needed to populate the yearly database (1403/1403 indicates the number of records processed). The times are in minutes and seconds:

- **2012**: 100% | 1403/1403 | Time: 32:11  
- **2013**: 100% | 1403/1403 | Time: 31:10  
- **2014**: 100% | 1403/1403 | Time: 29:53  
- **2015**: 100% | 1403/1403 | Time: 33:18  
- **2016**: 100% | 1403/1403 | Time: 33:10  
- **2017**: 100% | 1403/1403 | Time: 23:19  
- **2018**: 100% | 1403/1403 | Time: 22:20  



## I. Preparation
### Importing Libraries and Setting Up Metadata

This section of the code imports the necessary libraries and sets up variables required for populating the DuckDB database with economic data:

In [1]:
# Import necessary libraries
import requests as r  # Library for making HTTP requests to APIs
import pandas as pd  # Library for data manipulation and analysis, especially with DataFrames
import datetime  # Library for handling date and time operations
import db_zip_populator as zp  # Custom module for populating the database (assumed to be defined elsewhere)
import re  # Library for working with regular expressions

import sqlite3  # Library for interacting with SQLite databases
from tqdm import tqdm  # Library for displaying progress bars during iterations
import duckdb  # Library for connecting to and manipulating DuckDB databases

# Set up variables
endyear = datetime.date.today().year  # Get the current year
api_headers = {}  # Initialize a dictionary to hold API headers
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'  # Set the API key for authentication in requests
db_path = '../zip_data/duck_db_manager/database/us_naics_census_data'  # Default Path to the DuckDB database file

/Users/nivyni/Documents/Work_SDE/ModelEarth/WorkingRepos/ModelEarth-data-pipeline/industries/naics/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## II. Retrieve Zipcode data
- The ZipCodeUtility class has 'startyear' and 'endyear' as parameters. If you need to download new data, you can adjust the start and end years. 

- There is a possibility that this won't work if the api call structure is changed. If that's the case, adjusting the '_get_response_data' function in db_zip_populator.py will do the trick.

In [2]:
# To update the database, specify the 'startyear' and 'endyear' parameters.
# For example, setting both parameters to 2024 will retrieve data for that year.
# zip_util = zu.ZipCodeUtility(industry_levels=[2, 5, 6], api_headers=api_headers, startyear=2024, endyear=2024)
# Please specify the NAICS levels for the database. The commented example reflects the current database configuration.
zip_populator = zp.ZipPopulator(industry_levels=[2, 5, 6], db_path=db_path, api_headers=api_headers, startyear=2012, endyear=2023, separate_databases=True)


### II.(a) Retrieve Specific Annual Data
- important: Only run this code if you try to populate a new annual database. If there's existing database in the path, please delete it if you want to repopulate it. 

In [ ]:
year = 2023 # Replace it with the desired year.

In [ ]:
zip_populator.get_zip_for_year(year)

### II.(b) Retrieve All Annual Data Upto Current Year (Not Including)
- important: only run this code if you try to reconstruct all databases. it could take 30-60 min * {num_years} overall time. And as before, please ensure there's no exsiting databaases in the desried path. 

In [ ]:
for i in range(2012, endyear):
     zip_populator.get_zip_for_year(i)

## III. Verify Retrieval Results
-  Check if the Separate Annual DuckDB Databases is equal to a single database
-  The route for the single database is inclued, please download the file and put it into the same folder as other separate annual databases for verification only. 

In [38]:
year = 2012 # Replace it with the year you want to check 
db_to_check = f"{db_path}_{year}.duckdb"

In [4]:
db_path_single = '../zip_data/duck_db_manager/database/us_economic_data_og.duckdb' # Replace it with the actual path to the database as reference

Compare Separate Annual Databases and One Single Database 

### (a). Check if the content of DimYear, DimNaics, DimZipCode are the same:

In [92]:
import duckdb

def compare_databases(db1, db2):
    # Connect to both databases
    conn1 = duckdb.connect(db1)
    conn2 = duckdb.connect(db2)

    try:
        # Fetch data from DimYear
        dim_year_db1 = set(conn1.execute("SELECT * FROM DimYear").fetchall())
        dim_year_db2 = set(conn2.execute("SELECT * FROM DimYear").fetchall())

        # Compare DimYear
        print("DimYear Differences:")
        only_in_db1 = dim_year_db1 - dim_year_db2
        only_in_db2 = dim_year_db2 - dim_year_db1
        print("Only in DB1:", only_in_db1)
        print("Only in DB2:", only_in_db2)

        # Fetch data from DimNaics
        dim_naics_db1 = set(conn1.execute("SELECT * FROM DimNaics").fetchall())
        dim_naics_db2 = set(conn2.execute("SELECT * FROM DimNaics").fetchall())

        # Compare DimNaics
        print("\nDimNaics Differences:")
        only_in_db1 = dim_naics_db1 - dim_naics_db2
        only_in_db2 = dim_naics_db2 - dim_naics_db1
        print("Only in DB1:", only_in_db1)
        print("Only in DB2:", only_in_db2)

        # Fetch data from DimZipCode
        dim_zipcode_db1 = set(conn1.execute("SELECT * FROM DimZipCode").fetchall())
        dim_zipcode_db2 = set(conn2.execute("SELECT * FROM DimZipCode").fetchall())

        # Compare DimZipCode
        print("\nDimZipCode Differences:")
        only_in_db1 = dim_zipcode_db1 - dim_zipcode_db2
        only_in_db2 = dim_zipcode_db2 - dim_zipcode_db1
        print("Only in DB1:", only_in_db1)
        print("Only in DB2:", only_in_db2)

    except Exception as e:
        print(f"Error during comparison: {e}")
    finally:
        # Close connections
        conn1.close()
        conn2.close()

# Example usage
compare_databases(db_to_check, db_path_single)

DimYear Differences:
Only in DB1: set()
Only in DB2: set()

DimNaics Differences:
Only in DB1: set()
Only in DB2: set()

DimZipCode Differences:
Only in DB1: {('92509', 'Jurupa Valley', 'CA'), ('97250', 'Portland', 'OR'), ('85336', 'San Luis', 'AZ'), ('84059', 'Vineyard', 'UT'), ('92587', 'Menifee', 'CA'), ('92586', 'Menifee', 'CA'), ('92585', 'Menifee', 'CA'), ('76130', 'Fort Worth', 'TX'), ('08695', 'Trenton', 'NJ'), ('98170', 'Seattle', 'WA'), ('53558', 'Mcfarland', 'WI'), ('77428', 'Collegeport', 'TX'), ('63040', 'Wildwood', 'MO'), ('37934', 'Farragut', 'TN'), ('85144', 'San Tan Valley', 'AZ'), ('08691', 'Robbinsville', 'NJ'), ('63038', 'Wildwood', 'MO'), ('98424', 'Fife', 'WA'), ('33460', 'Lake Worth Beach', 'FL')}
Only in DB2: set()


### (b). Check if the content of 2012-2019 is the same as a single db (level 2 and level 6)

In [7]:
import duckdb
import pandas as pd

def compare_dataentry(db1, db2, years, industry_levels=[2, 6]):
    """
    Compare DataEntry for specified Industry Levels and multiple Years between two DuckDB databases
    and print a summary of the differences.

    Parameters:
    - db1: str, path to the first DuckDB database.
    - db2: str, path to the second DuckDB database.
    - years: list, list of years to filter entries.
    - industry_levels: list, list of industry levels to filter entries (default is [2, 6]).
    """
    results = {}

    for year in years:
        with duckdb.connect(db1) as conn1, duckdb.connect(f"{db2}_{year}.duckdb") as conn2:
            try:
                # Query DataEntry from DB1 for the specified Industry Levels and Year
                dataentry_db1 = conn1.execute(
                    f"SELECT GeoID, NaicsCode, Year, Establishments, Employees, Payroll, IndustryLevel "f"FROM DataEntry WHERE IndustryLevel IN ({', '.join(map(str, industry_levels))}) AND Year = ?",
                    [year]
                ).fetchall()

                # Query DataEntry from DB2 for the specified Industry Levels and Year
                dataentry_db2 = conn2.execute(
                    f"SELECT GeoID, NaicsCode, Year, Establishments, Employees, Payroll, IndustryLevel "f"FROM DataEntry WHERE IndustryLevel IN ({', '.join(map(str, industry_levels))}) AND Year = ?",
                    [year]
                ).fetchall()

                # Convert to sets for comparison
                set_db1 = set(dataentry_db1)
                set_db2 = set(dataentry_db2)

                # Compare DataEntry
                only_in_db1 = set_db1 - set_db2
                only_in_db2 = set_db2 - set_db1

                # Count differences
                count_db1 = len(only_in_db1)
                count_db2 = len(only_in_db2)

                # Summarize differences by industry level
                level_2_db1 = sum(1for entry in only_in_db1 if entry[-1] == 2)
                level_6_db1 = sum(1for entry in only_in_db1 if entry[-1] == 6)
                level_2_db2 = sum(1for entry in only_in_db2 if entry[-1] == 2)
                level_6_db2 = sum(1for entry in only_in_db2 if entry[-1] == 6)

                total_level_2_db1 = sum(1for entry in set_db1 if entry[-1] == 2)
                total_level_6_db1 = sum(1for entry in set_db1 if entry[-1] == 6)
                total_level_2_db2 = sum(1for entry in set_db2 if entry[-1] == 2)
                total_level_6_db2 = sum(1for entry in set_db2 if entry[-1] == 6)

                # Store results
                results[year] = {
                    'only_in_db1_total': count_db1,
                    'only_in_db1_level_2': level_2_db1,
                    'only_in_db1_level_6': level_6_db1,
                    'only_in_db2_total': count_db2,
                    'only_in_db2_level_2': level_2_db2,
                    'only_in_db2_level_6': level_6_db2,
                    'total_db1_level_2': total_level_2_db1,
                    'total_db1_level_6': total_level_6_db1,
                    'total_db2_level_2': total_level_2_db2,
                    'total_db2_level_6': total_level_6_db2
                }
            except Exception as e:
                print(f"Error during DataEntry comparison for year {year}: {e}")
                results[year] = None 
                # Print results in a DataFrame
        for year, data in results.items():
            if data:
                df = pd.DataFrame([{
                    'Database': ['Only in DB1', 'Only in DB2', 'Total in DB1', 'Total in DB2'],
                    'Total Count': [data['only_in_db1_total'], data['only_in_db2_total'], data['total_db1_level_2'] + data['total_db1_level_6'], data['total_db2_level_2'] + data['total_db2_level_6']],
                    'Level 2 Count': [data['only_in_db1_level_2'], data['only_in_db2_level_2'], data['total_db1_level_2'], data['total_db2_level_2']],
                    'Level 6 Count': [data['only_in_db1_level_6'], data['only_in_db2_level_6'], data['total_db1_level_6'], data['total_db2_level_6']]
                }], index=[''])
                
                print(f"\nComparison for Year {year}:")
                print(df.to_string(index=False))
            else:
                print(f"\nNo data available for Year {year}")


In [33]:
def compare_dataentry_countonly(db1, db2, years, industry_levels=[2, 6]):
    results = {}

    for year in years:
        with duckdb.connect(db1) as conn1, duckdb.connect(f"{db2}_{year}.duckdb") as conn2:
            try:
                # Query DataEntry from DB1
                dataentry_db1 = conn1.execute(
                    f"SELECT IndustryLevel, COUNT(*) AS Count "f"FROM DataEntry WHERE IndustryLevel IN ({', '.join(map(str, industry_levels))}) AND Year = ? "f"GROUP BY IndustryLevel",
                    [year]
                ).fetchall()

                # Query DataEntry from DB2
                dataentry_db2 = conn2.execute(
                    f"SELECT IndustryLevel, COUNT(*) AS Count "f"FROM DataEntry WHERE IndustryLevel IN ({', '.join(map(str, industry_levels))}) AND Year = ? "f"GROUP BY IndustryLevel",
                    [year]
                ).fetchall()

                # Convert results to dictionaries
                count_db1 = {row[0]: row[1] for row in dataentry_db1}
                count_db2 = {row[0]: row[1] for row in dataentry_db2}

                # Calculate differences
                difference = {}
                for level in industry_levels:
                    count1 = count_db1.get(level, 0)
                    count2 = count_db2.get(level, 0)
                    difference[level] = {
                        'Orignal Single DB': count1,
                        'New Annual DB': count2,
                        'Difference': count1 - count2
                    }

                # Store results
                results[year] = difference

            except Exception as e:
                print(f"Error during DataEntry comparison for year {year}: {e}")
                results[year] = None
    return results

    

In [11]:
# Define the range of years you want to compare
years = range(2017, 2019)

# Paths to your databases
db_path_single = '../zip_data/duck_db_manager/database/us_economic_data_og.duckdb'
db_path_separate = '../zip_data/duck_db_manager/database/us_naics_census_data'
compare_dataentry(db_path_single, db_path_separate, years)



Comparison for Year 2017:
                                              Database                  Total Count               Level 2 Count                Level 6 Count
[Only in DB1, Only in DB2, Total in DB1, Total in DB2] [958986, 0, 1907240, 948254] [494172, 0, 734100, 239928] [464814, 0, 1173140, 708326]


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Comparison for Year 2017:
                                              Database                  Total Count               Level 2 Count                Level 6 Count
[Only in DB1, Only in DB2, Total in DB1, Total in DB2] [958986, 0, 1907240, 948254] [494172, 0, 734100, 239928] [464814, 0, 1173140, 708326]

Comparison for Year 2018:
                                              Database                  Total Count               Level 2 Count                Level 6 Count
[Only in DB1, Only in DB2, Total in DB1, Total in DB2] [964153, 0, 1914735, 950582] [495976, 0, 736350, 240374] [468177, 0, 1178385, 710208]


In [39]:
# Define the range of years you want to compare
years = range(2017, 2019)

# Paths to your databases
db_path_single = '../zip_data/duck_db_manager/database/us_economic_data_og.duckdb'
db_path_separate = '../zip_data/duck_db_manager/database/us_naics_census_data'


In [40]:
result = compare_dataentry_countonly(db_path_single, db_path_separate, years)

# Print results in a pretty format
for year, data in result.items():
    if data is None:
        print(f"Comparison for Year {year}: Error")
        continue
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index.name = 'Industry Level'
    print(f"Comparison for Year {year}:")
    print(df)
    print()

Comparison for Year 2017:
                Orignal Single DB  New Annual DB  Difference
Industry Level                                              
2                          779292         239928      539364
6                         1326547         708326      618221

Comparison for Year 2018:
                Orignal Single DB  New Annual DB  Difference
Industry Level                                              
2                          782345         240374      541971
6                         1331948         710208      621740



### Detailed check for db 2017,2018, which have different values

In [15]:
import duckdb

def summarize_difference_between_dbs(db1_path, db2_path, year, industry_levels=[2, 6]):
    """
    Summarize differences in entries between two DuckDB databases for a given year and industry levels.

    Parameters:
    - db1_path: str, path to the first DuckDB database.
    - db2_path: str, path to the second DuckDB database.
    - year: int, the year to filter entries.
    - industry_levels: list, list of industry levels to filter entries (default is [2, 6]).

    Returns:
    - summary: DataFrame, summary of differences between the two databases.
    """
    with duckdb.connect(db1_path) as conn1, duckdb.connect(db2_path) as conn2:
        # Query counts for db1
        query_db1 = f"""
            SELECT IndustryLevel, COUNT(*) as entry_count
            FROM DataEntry
            WHERE Year = ? AND IndustryLevel IN ({', '.join(map(str, industry_levels))})
            GROUP BY IndustryLevel
        """
        summary_db1 = conn1.execute(query_db1, [year]).fetchdf()

        # Query counts for db2
        query_db2 = f"""
            SELECT IndustryLevel, COUNT(*) as entry_count
            FROM DataEntry
            WHERE Year = ? AND IndustryLevel IN ({', '.join(map(str, industry_levels))})
            GROUP BY IndustryLevel
        """
        summary_db2 = conn2.execute(query_db2, [year]).fetchdf()

        # Merge the summaries
        summary = summary_db1.set_index('IndustryLevel').join(
            summary_db2.set_index('IndustryLevel'), 
            lsuffix='_db1', 
            rsuffix='_db2', 
            how='outer'
        ).fillna(0)

        # Calculate differences
        summary['Difference'] = summary['entry_count_db2'] - summary['entry_count_db1']
    
    return summary.reset_index()




In [17]:
# Paths to your DuckDB databases
db1_path = db_path_single
db2_path = db_path_separate

for year in range(2017, 2020):
    summary = summarize_difference_between_dbs(db_path_single, f"{db_path_separate}_{year}.duckdb", year)
    print(f"Differences between DB1 and DB2 for {year}:", year)
    print(summary)


Differences between DB1 and DB2 for 2017: 2017
   IndustryLevel  entry_count_db1  entry_count_db2  Difference
0              2           779292           239928     -539364
1              6          1326547           708326     -618221
Differences between DB1 and DB2 for 2018: 2018
   IndustryLevel  entry_count_db1  entry_count_db2  Difference
0              2           782345           240374     -541971
1              6          1331948           710208     -621740
Differences between DB1 and DB2 for 2019: 2019
   IndustryLevel  entry_count_db1  entry_count_db2  Difference
0              2           239836           239836           0
1              6           713776           713776           0


In [20]:
import duckdb

def summarize_difference_between_dbs(db1_path, db2_path, year, industry_levels=[2, 6]):
    """
    Summarize differences in entries between two DuckDB databases for a given year and industry levels.

    Parameters:
    - db1_path: str, path to the first DuckDB database.
    - db2_path: str, path to the second DuckDB database.
    - year: int, the year to filter entries.
    - industry_levels: list, list of industry levels to filter entries (default is [2, 6]).

    Returns:
    - summary: DataFrame, summary of differences between the two databases.
    """
    with duckdb.connect(db1_path) as conn1, duckdb.connect(db2_path) as conn2:
        # Query counts for db1
        query_db1 = f"""
            SELECT NaicsCode, IndustryLevel, COUNT(*) as entry_count
            FROM DataEntry
            WHERE Year = ? AND IndustryLevel IN ({', '.join(map(str, industry_levels))})
            GROUP BY NaicsCode, IndustryLevel
        """
        summary_db1 = conn1.execute(query_db1, [year]).fetchdf()

        # Query counts for db2
        query_db2 = f"""
            SELECT NaicsCode, IndustryLevel, COUNT(*) as entry_count
            FROM DataEntry
            WHERE Year = ? AND IndustryLevel IN ({', '.join(map(str, industry_levels))})
            GROUP BY NaicsCode, IndustryLevel
        """
        summary_db2 = conn2.execute(query_db2, [year]).fetchdf()

        # Merge the summaries
        summary = summary_db1.set_index(['NaicsCode', 'IndustryLevel']).join(
            summary_db2.set_index(['NaicsCode', 'IndustryLevel']), 
            lsuffix='_db1', 
            rsuffix='_db2', 
            how='outer'
        ).fillna(0)

        # Calculate differences
        summary['Difference'] = summary['entry_count_db2'] - summary['entry_count_db1']
    
    return summary.reset_index()


In [21]:
for year in range(2017, 2020):
    summary = summarize_difference_between_dbs(db_path_single, f"{db_path_separate}_{year}.duckdb", year)
    print(f"Differences between DB1 and DB2 for {year}:", year)
    print(summary)

Differences between DB1 and DB2 for 2017: 2017
    NaicsCode  IndustryLevel  entry_count_db1  entry_count_db2  Difference
0          00              2           159678            35279     -124399
1          11              2             4209             2422       -1787
2      113110              6                8                5          -3
3      113310              6             1387              831        -556
4      114111              6              155               78         -77
..        ...            ...              ...              ...         ...
886    813920              6              952              560        -392
887    813930              6             2385             1503        -882
888    813940              6              247              135        -112
889    813990              6             3239             1696       -1543
890        99              2              929              486        -443

[891 rows x 5 columns]
Differences between DB1 and D

In [22]:
def count_entries(db_path):
    """
    Count the total number of entries in the DataEntry table of the specified DuckDB database.

    Parameters:
    - db_path: str, path to the DuckDB database.

    Returns:
    - entry_count: int, total number of entries in the DataEntry table.
    """
    with duckdb.connect(db_path) as conn:
        query = "SELECT COUNT(*) AS entry_count FROM DataEntry"
        entry_count = conn.execute(query).fetchone()[0]  # Fetch the count from the result
    return entry_count

In [24]:
for year in range(2012, 2020):
    count = count_entries(f"{db_path_separate}_{year}.duckdb")
    print(f"Entries for Year {year}",year)
    print(count)

Entries for Year 2012 2012
5569846
Entries for Year 2013 2013
5565493
Entries for Year 2014 2014
5570969
Entries for Year 2015 2015
5591817
Entries for Year 2016 2016
5604918
Entries for Year 2017 2017
1640276
Entries for Year 2018 2018
1643667
Entries for Year 2019 2019
1649947


### Finding the GeoID (ZipCode) Value of 99999

#### 1. Single Database

In [2]:
db_path_single = '../zip_data/duck_db_manager/database/us_economic_data_og.duckdb'

In [8]:
import duckdb
import pandas as pd
from IPython.display import display

# Create a connection to the DuckDB database
conn = duckdb.connect(database=db_path_single)

# Initialize a dictionary to store counts for each year and industry level
counts = {}

# Iterate over the years from 2012 to 2023
for year in range(2012, 2024):
    # Query to count the number of 99999 GeoID entries for the current year, grouped by IndustryLevel
    query = f"""
            SELECT IndustryLevel, COUNT(*) AS count
            FROM DataEntry
            WHERE GeoID = '99999' AND Year = {year}
            GROUP BY IndustryLevel
            """# Execute the query and fetch the result
    result = conn.execute(query).fetchall()
    
    # Store the result in the dictionary
    counts[year] = result

# Flatten the counts dictionary to create a DataFrame
data = []
for year, levels in counts.items():
    for level, count in levels:
        data.append({'Year': year, 'IndustryLevel': level, 'Count': count})

df_counts = pd.DataFrame(data)

# Calculate the total count
total_count = df_counts['Count'].sum()

# Display the DataFrame using Jupyter's display function for a prettier output
display(df_counts)

# Print the total count separatelyprint(f"Total count of 99999 GeoID entries from 2012 to 2023: {total_count}")

# Close the database connection
conn.close()


,Year,IndustryLevel,Count
0,2012,2,18
1,2012,4,129
2,2012,6,278
3,2013,2,18
4,2013,4,129
5,2013,6,276
6,2014,2,17
7,2014,4,141
8,2014,6,297
9,2015,2,18


#### 2. Separate Annual Database

In [ ]:
db_path_separate = '../zip_data/duck_db_manager/database/us_naics_census_data'

In [ ]:
# Initialize a dictionary to store counts for each year and industry level


# Iterate over the years from 2012 to 2023
for year in range(2012, 2024):
    counts = {}
    conn = duckdb.connect(database=(f"{db_path_separate}_{year}.duckdb", year))
    
    # Query to count the number of 99999 GeoID entries for the current year, grouped by IndustryLevel
    query = f"""
            SELECT IndustryLevel, COUNT(*) AS count
            FROM DataEntry
            WHERE GeoID = '99999' AND Year = {year}
            GROUP BY IndustryLevel
            """# Execute the query and fetch the result
    result = conn.execute(query).fetchall()
    
    # Store the result in the dictionary
    counts[year] = result
    # Flatten the counts dictionary to create a DataFrame
    data = []
    for year, levels in counts.items():
        for level, count in levels:
            data.append({'Year': year, 'IndustryLevel': level, 'Count': count})

    df_counts = pd.DataFrame(data)

    # Calculate the total count
    total_count = df_counts['Count'].sum()

    # Display the DataFrame using Jupyter's display function for a prettier output
    display(df_counts)

    # Print the total count separatelyprint(f"Total count of 99999 GeoID entries from 2012 to 2023: {total_count}")

    # Close the database connection
    conn.close()

